# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686611


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   9%|▉         | 3/32 [00:02<00:25,  1.12it/s]

Rendering models:  12%|█▎        | 4/32 [00:02<00:18,  1.53it/s]

Rendering models:  25%|██▌       | 8/32 [00:02<00:11,  2.12it/s]

Rendering models:  28%|██▊       | 9/32 [00:05<00:22,  1.01it/s]

...

Rendering models:  78%|███████▊  | 25/32 [00:07<00:01,  6.06it/s]

Rendering models:  84%|████████▍ | 27/32 [00:08<00:00,  6.67it/s]

Rendering models:  94%|█████████▍| 30/32 [00:08<00:00,  8.66it/s]

Rendering models: 100%|██████████| 32/32 [00:08<00:00,  8.86it/s]

not-logged-in-bc5fdb9ea8d611e60b8b    0.000881
Almudys                               0.001290
Camelron                              0.000663
shocko61                              0.000499
shocko61                              0.000471
not-logged-in-83c90d58182a261c0405    0.005660
kjkavene                              0.000774
not-logged-in-c3e2bb63f9ec3bdeea91    0.000501
not-logged-in-f65c325548a8d20ddd46    0.001715
lsautter                              0.000490
not-logged-in-a92559927bfd05e023e7    0.001929
clars915                              0.000532
froggler                              0.000517
acapirala                             0.000564
Lavadude                              0.024852
ts135992                              0.000551
not-logged-in-8700ff8251e0704f745c    0.000510
not-logged-in-0582d7599288fe425356    0.000714
a-ikiso                               0.000990
r.sundstrom3                          0.000547
Daylyn                                0.000918
dhaywoo      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())